In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
header_names = ['timestamp', 'value', 'is_Anomaly']

In [5]:
dataset=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/disease_person_edit.csv',  names=header_names)

In [6]:
dataset

,timestamp,value,is_Anomaly
0,05:57.0,433,0.0
1,05:57.0,430,0.0
2,05:57.0,427,0.0
3,05:57.0,426,0.0
4,05:57.0,427,0.0
...,...,...,...
9710,06:47.4,574,0.0
9711,06:47.4,560,0.0
9712,06:47.5,547,0.0
9713,06:47.5,529,0.0


In [7]:
dataset['is_Anomaly'] = dataset['is_Anomaly'].round()

In [8]:

dataset['timestamp'] = pd.to_datetime(dataset['timestamp'])
dataset['timestamp'].min(),dataset['timestamp'].max()

(Timestamp('2021-06-07 05:57:00'), Timestamp('2021-06-07 06:47:30'))

In [9]:
dataset

,timestamp,value,is_Anomaly
0,2021-06-07 05:57:00,433,0.0
1,2021-06-07 05:57:00,430,0.0
2,2021-06-07 05:57:00,427,0.0
3,2021-06-07 05:57:00,426,0.0
4,2021-06-07 05:57:00,427,0.0
...,...,...,...
9710,2021-06-07 06:47:24,574,0.0
9711,2021-06-07 06:47:24,560,0.0
9712,2021-06-07 06:47:30,547,0.0
9713,2021-06-07 06:47:30,529,0.0


In [10]:
from sklearn.ensemble import IsolationForest
#to_model_column='value'
clf=IsolationForest(n_estimators=10, max_samples='auto', contamination=float(.04), \
                        max_features=1.0, bootstrap=False, n_jobs=-1, random_state=42, verbose=0)

In [11]:
clf.fit(dataset[['value']])

IsolationForest(behaviour='deprecated', bootstrap=False, contamination=0.04,
                max_features=1.0, max_samples='auto', n_estimators=10,
                n_jobs=-1, random_state=42, verbose=0, warm_start=False)

In [12]:
dataset['scores']=clf.decision_function(dataset[['value']])
dataset['anomaly']=clf.predict(dataset[['value']])


In [13]:
dataset.loc[dataset['anomaly'] == 1,'anomaly'] = 0
dataset.loc[dataset['anomaly'] == -1,'anomaly'] = 1

In [14]:
dataset

,timestamp,value,is_Anomaly,scores,anomaly
0,2021-06-07 05:57:00,433,0.0,0.238437,0
1,2021-06-07 05:57:00,430,0.0,0.223367,0
2,2021-06-07 05:57:00,427,0.0,0.241614,0
3,2021-06-07 05:57:00,426,0.0,0.241614,0
4,2021-06-07 05:57:00,427,0.0,0.241614,0
...,...,...,...,...,...
9710,2021-06-07 06:47:24,574,0.0,0.211586,0
9711,2021-06-07 06:47:24,560,0.0,0.209455,0
9712,2021-06-07 06:47:30,547,0.0,0.228495,0
9713,2021-06-07 06:47:30,529,0.0,0.241583,0


In [15]:
from sklearn.metrics import accuracy_score

accuracy_score(dataset['is_Anomaly'], dataset['anomaly'])

0.9565620174987133

In [16]:
from sklearn.metrics import confusion_matrix
confusion_matrix(dataset['is_Anomaly'], dataset['anomaly'])

array([[9291,  373],
       [  49,    2]])

In [17]:
from sklearn.metrics import classification_report
print(classification_report(dataset['is_Anomaly'], dataset['anomaly']))

              precision    recall  f1-score   support

         0.0       0.99      0.96      0.98      9664
         1.0       0.01      0.04      0.01        51

    accuracy                           0.96      9715
   macro avg       0.50      0.50      0.49      9715
weighted avg       0.99      0.96      0.97      9715



In [19]:
anomaly=dataset.loc[dataset['anomaly']==1]
anomaly_index=list(anomaly.index)
print(anomaly)

               timestamp  value  is_Anomaly    scores  anomaly
94   2021-06-07 05:57:30    949         1.0 -0.047279        1
95   2021-06-07 05:57:30    931         1.0 -0.018529        1
255  2021-06-07 05:58:24    931         0.0 -0.018529        1
256  2021-06-07 05:58:24    934         0.0 -0.018529        1
851  2021-06-07 06:01:30    912         0.0 -0.013849        1
...                  ...    ...         ...       ...      ...
8513 2021-06-07 06:41:12    752         0.0 -0.003082        1
9059 2021-06-07 06:44:06    929         0.0 -0.018529        1
9060 2021-06-07 06:44:06    949         0.0 -0.047279        1
9227 2021-06-07 06:44:54    751         0.0 -0.003082        1
9663 2021-06-07 06:47:12    908         0.0 -0.013849        1

[375 rows x 5 columns]
